# 실습 WEEK10: Numpy

:::{admonition} 안내
:class: info  

- 실습 문제는 사전, 도전, 완전 실습 문제로 구성되어 있습니다.
- 사전 실습 문제는 수업 전에 미리 풀어보시면 도움이 됩니다.
- **<font color='blue'>도전 실습 문제는 etl>우리강좌>코딩수업>에 접속한 후, 해당 주차 문제의 소스코드를 제출하여 테스트를 통과해야 과제 점수를 받을 수 있습니다.</font>**
- **<font color='red'>제출기한은 코딩수업(엘리스코딩)의 해당 주차 응시 기간을 확인해주세요.</font>**
- 완전 실습 문제는 보너스 실습 문제로 생각하시면 됩니다.  도전 실습 문제를 다 풀어본 학생들이 추가적으로 풀어보시면 도움이 됩니다.
- 사전 실습 문제와 완전 실습 문제는 수업 시간에 별도 풀이를 하지 않습니다.
- 질문은 사전, 도전, 완전 그리고 수업과 관련된 모든 것에  대하여 교수자 or 조교 or 튜터에게 하시면 됩니다.
- **<font color='red'>가독성을 위해 입출력 데이터에서 콜론(:) 다음에는 항상 한칸 공백이 있습니다.</font>**

:::

## [도전 실습 문제]

##Problem 1:(함께) 배열생성
😄 다음과 같은 결과가 나오도록 배열을 생성하시오.
```
[[10 11 12 13 14]
 [15 16 17 18 19]
 [20 21 22 23 24]]
 ```

- numpy 모듈의 `array()`를 통해 배열을 만들 수 있다.
- numpy 모듈의 `arange()`를 통해 배열을 만들 수 있다.


```
np.array(리스트 or range())
np.arange(start, end, step)
```



## Problem 2:(스스로) 배열생성
😄 다음과 같은 결과가 나오도록 배열을 생성하시오.
```
[[[ 74  79  84  89]
  [ 94  99 104 109]
  [114 119 124 129]]

 [[134 139 144 149]
  [154 159 164 169]
  [174 179 184 189]]

 [[194 199 204 209]
  [214 219 224 229]
  [234 239 244 249]]]
 ```

##Problem 3:(함께) 랜덤데이터 배열생성
😄 다음과 같이 10이상 20미만의 임의의 정수로 채운 배열을 생성하시오.
```
[[[14 19 11 15 12 12 13 16 11]
  [12 14 18 15 11 14 18 10 10]
  [11 18 18 11 19 10 12 17 19]]

 [[13 17 17 15 18 14 16 14 13]
  [16 17 12 13 18 18 15 14 12]
  [15 18 15 16 18 16 12 18 19]]

 [[18 11 12 13 10 15 11 18 10]
  [15 16 15 15 15 12 17 11 11]
  [14 10 11 11 10 19 18 19 15]]]
```

### np.random
- 0 ≤ 실수 < 1로 채우기: np.random.random(배열의크기)
- a ≤ 실수 < b로 채우기: np.random.uniform(a, b, 배열의크기)
- a ≤ 정수 < b로 채우기: np.random.randint(a, b, 배열의크기)

##Problem 4:(스스로) 랜덤데이터 배열생성
😄 다음과 같이 10이상 20미만의 임의의 실수(소수점 2자리)로 채운 배열을 생성하시오.
```
[[[11.38 18.23 10.57 16.33 13.  ]
  [14.5  16.85 16.13 17.97 17.51]]

 [[18.49 12.98 16.6  13.02 16.62]
  [13.37 19.94 16.39 11.79 11.96]]

 [[12.02 11.33 13.82 16.63 17.84]
  [17.78 14.46 18.83 11.22 13.97]]]
```

##Problem 5:(함께) 균등분할 배열생성
😄 110이상 120이하의 수를 5개 균등 분할하는 배열을 생성하시오.
```
[110.  112.5 115.  117.5 120. ]
```

### linspace()
- linspace(): 일정한 간격으로 등분한 배열을 생성한다.

```
np.linspace(start,end,개수)
```

##Problem 6:(함께) 전치행렬
😄 다음과 같은 배열을 생성해 보고, 그 배열을 전치하시오.
```
[[ 0  1  2  3  4]
 [ 5  6  7  8  9]
 [10 11 12 13 14]]
```

```
[[ 0  5 10]
 [ 1  6 11]
 [ 2  7 12]
 [ 3  8 13]
 [ 4  9 14]]
 ```

### transpose()
- transpose(): 행과 열을 교체한다.  `배열명.T`도 가능하다.

```
np.transpose(배열명)
배열명.T
```

##Problem 7:(함께) 배열연산
😄 앞의 배열에서 뒤의 배열을 뺀 뒤, 각 값이 양수인지 아닌지를 True, False로 출력하시오.
```
[[ 7  5  9 10]
 [ 4 10  6  2]]
```
```
[[ 1  3  4  9]
 [14  7  6  4]]
```
```
[[ True  True  True  True]
 [False  True False False]]
```

##Problem 8:(함께) 최대값, 최소값, 위치
😄 다음과 같이 10이상 20미만의 임의의 실수(소수점 2자리)로 채운 배열에서 최대값 과 최소값을 찾아보고, 최대값과 최소값의 위치를 출력 하시오.
```
[[[11.23 16.13 14.11 15.19 12.18]
  [17.47 19.86 15.46 14.13 13.72]]

 [[18.26 11.6  10.11 18.8  12.79]
  [10.18 12.85 14.11 12.32 14.76]]

 [[11.99 19.75 19.7  11.91 14.94]
  [17.09 14.13 16.94 13.79 10.4 ]]]
```
```
최소값: 10.11
최대값: 19.86
최소값 위치: 12
최대값 위치: 6
```

### min(), max(), argmin(), argmax()
- np.min(): 최소값을 반환한다.
- np.max(): 최대값을 반환한다.
- argmin(): 최소값 인덱스를 반환한다.
- argmax(): 최대값 인덱스를 반환한다.

## Problem 9:(스스로) 대통령 평균 신장 구하기
- 배열에서 다음을 구하시오.

```
Mean height: 180.05
Standard deviation: 6.98
Minimum height: 163.0
Maximum height: 193.0
```


In [ ]:
ar = np.array([189., 170., 189., 163., 183., 171., 185., 168., 173., 183., 173.,
       173., 175., 178., 183., 193., 178., 173., 174., 183., 183., 168.,
       170., 178., 182., 180., 183., 178., 182., 188., 175., 179., 183.,
       193., 182., 183., 177., 185., 188., 188., 182., 185., 191., 182.])

## Problem 10: (스스로)가중치 적용하여 성적구하기
- 다음과 같은 배열이 있다.
- 자료는 앞에서부터 학번, 국어, 영어, 수학, 체육, 컴퓨터의 시험점수이다.
1. 전체 평균점수가 가장 높은 학생의 번호는?
2. 전체 평균점수가 가장 낮은 학생의 번호는?
3. 국어, 영어, 수학 점수의 평균이 가장 높은 학생의 번호는?
4. 국어, 영어, 수학 점수에 2배 가중치를 준다면 5과목의 평균 점수가 가장 높은 학생의 번호는?

Output Samples
1. 20232
2. 20237
3. 20232
4. 20232

In [ ]:
import numpy as np
scores=np.array([[20231., 70., 85., 95., 100., 100.],
          [20232., 100., 85., 90., 90., 95.],
          [20233., 95., 95., 80., 75., 75.],
          [20234., 65., 75., 70., 90., 100.],
          [20235., 90., 80., 85., 95., 95.],
          [20236., 90., 90., 75., 75., 80.],
          [20237., 60., 90., 80., 40., 45.],
          [20238., 85., 90., 75., 60., 90.],
          [20239., 90., 85., 85., 90., 85.]])
scores